# Python with APIs
Exchanges via HTTP using `requests` package

In [10]:
import requests

## Example

### Agify API

Example route for Agify API

In [11]:
agify_dan = "https://api.agify.io/?name=dan"

Extract response

In [12]:
# Run a GET request
agify_answer = requests.get(agify_dan)
agify_answer_txt = agify_answer.text
agify_answer_json = agify_answer.json()

print(f"Answer status_code: {agify_answer}")

print(type(agify_answer_txt))
print(type(agify_answer_json))

print(agify_answer_txt)
print(agify_answer_json)

Answer status_code: <Response [429]>
<class 'str'>
<class 'dict'>
{"error":"Request limit reached"}
{'error': 'Request limit reached'}


## Introductory Exercises

In [13]:
# Exercise 1
# Write a script that asks the user for their first name
# and responds with a personalized message
# using the agify API

# https://api.agify.io/?name=YOUR_NAME
# Example response: {"name":"YOUR_NAME","age":30,"count":12345}

def get_data(name,api):
    response = requests.get(f"{api},{name}")
    if response.status_code == 200:
        return response.json()
    else:
        return None

user_name = input("Please enter your first name: ")
data = get_data(user_name,"https://api.agify.io/?name=")

if data:
    estimated_age = data['age']
    count = data['count']
    print("Your name is ",user_name,", your age is ",estimated_age,"and you are ",estimated_age, " in the world")
else:
    print("Fail")


In [14]:
# Exercise 2
# Write a script that asks the user for their first name
# and responds with a personalized message
# using the genderize API
# https://api.genderize.io/?name=YOUR_NAME

user_name = input("Please enter your first name: ")
data = get_data(user_name,"https://api.genderize.io/?name=")

if data:
    estimated_gender = data['gender']
    proba = data['probability'] * 100
    print(f"With the name {user_name}, the probability of being {estimated_gender} is {proba}%")
else:
    print("Fail")


In [15]:
# Exercise 3
# Write a script that asks the user for their first name
# and responds with a personalized message
# using the nationalize API
# https://api.nationalize.io/?name=YOUR_NAME

user_name = input("Please enter your first name: ")
data = get_data(user_name, "https://api.nationalize.io/?name=")

if data and 'country' in data:
    print(f"Based on your name, {user_name}, here are your top estimated nationalities and their probabilities:")
    for country in data['country']:
        country_id = country['country_id']
        proba = country['probability'] * 100
        print(f"{country_id}: {proba:.2f}%")
else:
    print("Failed to fetch estimated nationality data.")


In [16]:
# Exercise 3.1
# Parsing the response from the nationalize API
# Get the most probable country and its percentage
# Example response: {"name":"YOUR_NAME","country":[{"country_id":"FR","probability":0.75},{"country_id":"BE","probability":0.25}]}
# Hint: use the max() function with a lambda function
# https://docs.python.org/3/library/functions.html#max
# https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions

user_name = input("Please enter your first name: ")
data = get_data(user_name, "https://api.nationalize.io/?name=")

if data and 'country' in data and data['country']:
    most_probable = max(data['country'], key=lambda country: country['probability'])
    country_id = most_probable['country_id']
    proba = most_probable['probability'] * 100
    print(f"Based on your name, {user_name}, the most probable nationality is {country_id} with a probability of {proba:.2f}%.")
else:
    print("Failed to fetch estimated nationality data or no data available.")



In [17]:
# Exercise 4
# Use BoredAPI : https://www.boredapi.com/
# Documentation : https://www.boredapi.com/documentation

# 1. Write a script that generates random activities
response = requests.get("https://www.boredapi.com/api/activity")
if response.status_code == 200:
  if response.json():
      print(f"Q1. Activity: {response.json()['activity']}")
  else:
      print("Q1. Failed to fetch activity.")


# 3. Write a script that generates random activities for 4 participants
response = requests.get(f"https://www.boredapi.com/api/activity?participants=4")
if response.status_code == 200:
  if response.json():
      print(f"Q3. Activity for 4 participants: {response.json()['activity']}")
  else:
      print("Q3. Failed to fetch activity.")


# 4. Write a script that generates random activities for 4 participants and of type "recreational"

response = requests.get(f"https://www.boredapi.com/api/activity?participants=4&type=recreational")
if response.status_code == 200:
  if response.json():
      print(f"Q4. Recreational activity for 4 participants: {response.json()['activity']}")
  else:
      print("Q4. Failed to fetch activity.")


# 5. Write a script that generates random activities for 2 participants and that does not require equipment
response = requests.get(f"https://www.boredapi.com/api/activity?participants=2")
if response.status_code == 200:
  if response.json():
      print(f"Q5. Activity for 2 participants: {response.json()['activity']}")
  else:
      print("Q5. Failed to fetch activity.")


## Intermediate exercises

In [18]:
# OpenDomesday
# https://opendomesday.org/api/

# Exercise 1
# Write a script that displays all the counties  
# using the OpenDomesday API.

response = requests.get("https://opendomesday.org/api/1.0/county/")
if response.status_code == 200:
  if response.json():
      print("Counties from OpenDomesday:")
      for county in response.json():
          print(county['name'])  
  else :
      print("Failed to fetch counties.")


In [19]:
# Exercise 2
# Write a script that displays the information
# of the county "Derbyshire".

if response.status_code == 200:
    counties = response.json()
    for county in counties:
        if "Derbyshire".lower() == county.get('name', '').lower():
            if county:
                print(county)
            else:
                print(f"Failed to fetch information.")


In [20]:
# Exercise 3
# Now that we have the ids for all the places in Derbyshire, we can load all their details...
# And from their details, we can list all the details of their manors.
# Go fetch the data!
# P.S.: remember to save the data to avoid downloading it every time

places_cache = {}
response = requests.get("https://opendomesday.org/api/1.0/county/dby/")

if response.status_code == 200:
    derbyshire_places = response.json()['places_in_county']  

    for place in derbyshire_places:
        place_id = place['id']
        
        if place_id not in places_cache:
            place_response = requests.get(f"https://opendomesday.org/api/1.0/place/{place_id}")
            if place_response.status_code == 200:
                place_details = place_response.json()
                places_cache[place_id] = place_details
            else:
                print(f"Failed to fetch details for place ID: {place_id}")
                continue
        else:
            place_details = places_cache[place_id]

        if 'manors' in place_details:
            for manor in place_details['manors']:
                print(f"Manor details: {manor}")
else:
    print("Failed to fetch places for Derbyshire.")


In [21]:
# Exercise 4
# Now that we have a quantity of raw data, we will extract the interesting parts.  
# In our case, we want to count the money paid by each manor and compare it to the number of ploughs it has.  
# - Can you find the corresponding json fields?  
# - Then, you can list these numbers for each manor in Derbyshire.  
# - And format this in an appropriate comma-separated values (CSV) file.

import csv

csv_file = 'manors_data.csv'

if places_cache:
    first_manor = next(iter(places_cache.values()))
    headers = list(first_manor.keys())

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)

        writer.writeheader()

        for manor_details in places_cache.values():
            writer.writerow(manor_details)

    print("CSV file has been created with dynamic headers.")
else:
    print("No data available to write.")


In [22]:
# Exercise 5
# What is the richest manor in Derbyshire?

#######################################################
################## YOUR ANSWER HERE ##################
#######################################################


In [23]:
# Exercise 6
# Give the total value paid by Derbyshire.

#######################################################
################## YOUR ANSWER HERE ##################
#######################################################


In [24]:
# Exercise 7
# Create a Python class.  
# It must include all the previous functionalities.  
# Refactor your code to make it readable, efficient, and maintainable.

#######################################################
################## YOUR ANSWER HERE ##################
#######################################################


In [25]:
# Exercise 8 (optional)
# Add to your class a system for error handling.  
# It must manage the following errors:  
# - Connection error  
# - Parsing error  
# - Request error  
# - Response error  
# - Parameter error

#######################################################
################## YOUR ANSWER HERE ##################
#######################################################

